# Trabalho Prático 1 - Processamento de Linguagem Natural

**Discente**: Victor Hugo Martins

**Professor**: Adriano Veloso

## Objetivo Geral

Este projeto tem como objetivo produzir e avaliar modelos de linguagem neural. A qualidade desses modelos depende dos hiperparâmetros utilizados. A habilidade a ser exercitada neste trabalho prático é a escolha adequada dos hiperparâmetros de um modelo de linguagem. Portanto, mais especificamente, serão produzidos diversos modelos de linguagem, cada um deles obtidos com diferentes hiperparâmetros, e em seguida você deverá avaliar cada modelo através de uma aplicação de analogias (descrita a seguir). Por fim, você deverá concluir a respeito dos melhores hiperparâmetros para o modelo de linguagem. Os hiperparâmetros a serem considerados são:

* CBOW ou Skip-gram
* Tamanho da janela de contexto
* Tamanho do embedding
* Quantidade de iterações de treinamento.

Ao fim da execução, os resultados obtidos serão analisados.


# Implementação

## Corpus

Nessa etapa, ocorre o download do dataset usado e construção do corpus a partir dele.

Um *corpus* é um conjunto de textos que serve como base de dados para análise e modelagem linguística. Ele é criado para representar um idioma ou contexto específico e pode incluir palavras, frases, sentenças ou até mesmo documentos completos. Dentro da tarefa proposta, o corpus será utilizado para treinar os dados dos modelos *Word2Vec* com um vocabulário amplo.

O *corpus* utilizado é o **Text8**, que contém os primeiros 100 milhões de caracteres da *Wikipédia* em lingua inglessa (edição de 2001), sendo restrito a letras minúsculas e sem pontuação. Seu vocabulário compacto e cheio de significado o torna útil em tarefas não apenas de similaridade semântica, mas também modelagem de tópicos e classificação de textos. Seu tamanho o tornam menos custoso em termos computacionais.

In [ ]:
import os
import urllib.request
import zipfile

def download_text8():
    url = "http://mattmahoney.net/dc/text8.zip"
    local_filename = "text8.zip"

    if not os.path.exists("text8"):
        print("Baixando o dataset text8...")
        urllib.request.urlretrieve(url, local_filename)

        with zipfile.ZipFile(local_filename, 'r') as zip_ref:
            zip_ref.extractall(".")
        print("Dataset extraído com sucesso!")

        os.remove(local_filename)

download_text8()

In [ ]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import datapath
import numpy as np

def load_and_preprocess_corpus(filepath):
    with open(filepath, 'r') as file:
        corpus = file.read().splitlines()
    return [line.split() for line in corpus]

O arquivo questions-words.txt é amplamente utilizado como um benchmark para avaliar modelos de vetorização de palavras, como Word2Vec. Ele contém pares de palavras e relações semânticas ou sintáticas organizadas em categorias. A ideia é testar a capacidade do modelo de capturar relações entre palavras usando operações vetoriais.

O teste clássico é baseado na ideia de analogia de palavras, como:
"Rei está para Rainha assim como Homem está para Mulher".
Matematicamente, isso é expresso como:
vec
(
𝑟
𝑒
𝑖
)
−
vec
(
ℎ
𝑜
𝑚
𝑒
𝑚
)
+
vec
(
𝑚
𝑢
𝑙
ℎ
𝑒
𝑟
)
≈
vec
(
𝑟
𝑎
𝑖
𝑛
ℎ
𝑎
)

O modelo é avaliado em quão bem ele encontra word4 ao operar sobre os vetores de word1, word2 e word3.

In [ ]:
import os
from sklearn.decomposition import PCA
import requests

def download_analogy_file(analogy_path):
    url = "https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt"
    if not os.path.exists(analogy_path):
        print("Baixando o arquivo de analogias...")
        response = requests.get(url)
        with open(analogy_path, "wb") as f:
            f.write(response.content)
        print("Download concluído.")

In [ ]:
# Variáveis de arquivos gerais
corpus_path = "text8"
analogy_path = "questions-words.txt"
download_analogy_file(analogy_path)
corpus = load_and_preprocess_corpus(corpus_path)

## Avaliação

O intuito é avaliar as analogias em torno de sua distância. Distâncias menores demonstram maior similaridade entre as palavras. Portanto, quão menor a distância, melhor o modelo é avaliado.

Para avaliar analogias com distância mínima entre vetores em embeddings (e, intrisecamente, capturar relações semânticas e sintáticas), utiliza-se a geometria do espaço vetorial.

Essa abordagem funciona porque palavras relacionadas estão mais próximas no espaço, e a distância ou similaridade do cosseno identifica o vetor mais próximo do resultado da analogia.

In [ ]:
# Avaliar a distância média de analogias
def evaluate_distance(model, analogy_path, apply_pca_flag, pca_components=0):
    word_vectors = model.wv.vectors
    word_to_index = model.wv.key_to_index

    if apply_pca_flag:
        word_vectors = apply_pca(word_vectors, n_components=pca_components)

    analogy_file = datapath(analogy_path)
    total_distance = 0
    valid_pairs = 0

    with open(analogy_file, "r") as f:
        for line in f:
            if line.startswith(":"):  # ignore categories
                continue
            words = line.strip().split()
            if all(word in word_to_index for word in words):
                idx_a, idx_b, idx_c, idx_d = [word_to_index[word] for word in words]
                vec_a, vec_b, vec_c, vec_d = (
                    word_vectors[idx_a],
                    word_vectors[idx_b],
                    word_vectors[idx_c],
                    word_vectors[idx_d],
                )
                predicted_vec = vec_c + (vec_b - vec_a)
                distance = np.linalg.norm(predicted_vec - vec_d)
                total_distance += distance
                valid_pairs += 1

    average_distance = total_distance / valid_pairs if valid_pairs > 0 else float("inf")
    return average_distance

**Grid Search** é uma técnica para identificar a melhor combinação de hiperparâmetros para um modelo. A escolha adequada de hiperparâmetros é crucial porque afeta diretamente o desempenho do modelo. Para cada combinação de hiperparâmetros no espaço definido, o modelo é treinado e avaliado com os dados fornecidos. A cada execução, o desempenho do modelo é avaliado usando uma métrica específica, que será usada para comparar os resultados das diferentes combinações.

Após avaliar todas as combinações, o conjunto de hiperparâmetros que maximiza ou minimiza a métrica escolhida é identificado como o melhor.

Além de contribuir para essa identificação, o Grid Search também atuam como **validação cruzada**.

A grade de parâmetros escolhida está definida abaixo:

In [ ]:
# Grid de parâmetros
param_grid = {
    "model_type": ["Skip-gram", "CBOW"],
    "context_size": [2, 3, 4, 5],
    "embedding_dim": [5, 20, 25, 50, 100],
    "num_epochs": [3, 5, 7, 10],
    "apply_pca_flag": [True, False],
    "pca_components": [2, 3, 5],  # Número de componentes para PCA
    "workers": [3],  # Número de componentes para PCA
}

E sua implementação:

In [ ]:
from itertools import product
import pandas as pd

def grid_search_word2vec(corpus, analogy_path, param_grid):
    best_params = None
    best_score = float("inf")
    results = []
    trained_models = {}  # Cache para modelos já treinados
    evaluation_cache = {}  # Cache para resultados de avaliação

    for params in product(*param_grid.values()):
        model_type, context_size, embedding_dim, num_epochs, apply_pca_flag, pca_components, workers = params

        # Ignorar variações de PCA components se PCA não for aplicado
        if not apply_pca_flag:
            pca_components = None

        # Criar uma chave única para identificar o modelo e incluir workers
        model_key = (model_type, context_size, embedding_dim, num_epochs, workers)
        eval_key = (model_key, apply_pca_flag, pca_components)

        # Treinar o modelo apenas se ainda não foi treinado para esta combinação
        if model_key not in trained_models:
            print(f"Treinando modelo com parâmetros: {model_key}")
            trained_models[model_key] = train_word2vec(
                corpus, model_type, context_size, embedding_dim, num_epochs, workers
            )

        model = trained_models[model_key]

        # Avaliar apenas se a avaliação para essa configuração ainda não foi feita
        if eval_key not in evaluation_cache:
            avg_distance = evaluate_distance(model, analogy_path, apply_pca_flag, pca_components)
            evaluation_cache[eval_key] = avg_distance
        else:
            avg_distance = evaluation_cache[eval_key]

        results.append({
            "Modelo": model_type,
            "Contexto": context_size,
            "Dimensão Embedding": embedding_dim,
            "Épocas": num_epochs,
            "Workers": workers,
            "PCA Aplicado": apply_pca_flag,
            "Componentes PCA": pca_components if apply_pca_flag else "Não Aplicado",
            "Distância Média": avg_distance
        })

        if avg_distance < best_score:
            best_score = avg_distance
            best_params = params

        print(f"Parâmetros: {params}, Distância média: {avg_distance:.4f}")

    df_results = pd.DataFrame(results)
    grouped_results = df_results.sort_values(by=["Modelo", "PCA Aplicado", "Distância Média"])

    return best_params, grouped_results

Um detalhe sobre essa implementação é o uso de chaves que evitem que o mesmo modelo seja treinado mais de uma vez. Caso isso não fosse trato, essa situação ocorreria para o caso do PCA não ser utilizado, onde a quantidade de componentes referentes a ele seria irrelevante. Com a adição das chaves, o custo computacional é reduzido ao não calcular mais de uma vez um mesmo modelo (ou seja, um modelo definido pelas tuplas de parâmetros).

Nas seções a seguir, serão abordados os parâmetros escolhidos para o Grid.

## PCA

PCA é um método de normalização que busca reduzir a dimensionalidade dos dados com o objetivo de facilitar a visualização e generalização do modelo. Apesar dessa proposta, que viria a diminuir o desempenho computacional do modelo, é possível que a aplicação de PCA faça com que os dados percam informações significativas e com que o modelo falhe a identificar relações não lineares.

Tendo essa dúvida em mente, a normalização por PCA também foi adicionada como um dos "parâmetros" para o GridSearch (mais detalhes sobre esse método e o porquê de sua escolha vem a seguir), buscando comparar se, para a tarefa de analogias especificamente, é mais vantajoso usar ou não deste método.

In [ ]:
from sklearn.decomposition import PCA

# Reduzir dimensionalidade com PCA
def apply_pca(vectors, n_components):
    pca = PCA(n_components=n_components)
    reduced_vectors = pca.fit_transform(vectors)
    return reduced_vectors

## Skip-Gram e CBOW

Skip-Gram e CBOW (Continuous Bag of Words) são arquiteturas utilizadas em modelos de representações vetoriais densas, como Word2Vec, para aprender representações vetoriais de palavras. Apesar de ambos identificarem relações sintáticas e semânticas, de forma que a aplicação de ambos se torne possível em tarefas de previsão de analogias, eles possuem especificidades e objetivos distintos.

* **CBOW (Continuous Bag of Words)**

Prediz uma palavra central a partir do contexto que a cerca.

Prediz uma palavra-alvo com base no contexto (as palavras que a cercam). Dentre as suas vantagens, estão o treinamento rápido e a eficiência em conjunto de dados menores. Como, em sua implementação, faz uso das médias dos vetores do contexto, essa arquitetura pode ser menor eficaz em capturar palavras raras.

* **Skip-Gram**

Prediz as palavras do contexto com base em uma palavra-central.

Em comparação com o CBOW, esta arquitetura atua melhor na captura de palavras raras, por possuir a capacidade de aprender relações específicas e mais detalhes entre palavra(s) e contexto. Entretanto, seu treinamento se dá de forma mais lenta.

In [ ]:
from gensim.models import Word2Vec
from gensim.test.utils import datapath
import numpy as np

# Treinar o modelo Word2Vec
def train_word2vec(corpus, model_type, context_size, embedding_dim, num_epochs, num_workers):
    sg = 1 if model_type == "Skip-gram" else 0  # 1 para Skip-gram, 0 para CBOW
    model = Word2Vec(
        sentences=corpus,
        vector_size=embedding_dim,
        window=context_size,
        sg=sg,
        epochs=num_epochs,
        workers=num_workers
    )
    return model

## Busca pelos Resultados

Abaixo, a Grid Search é executada, a fim de identificar quais os melhores (e piores) hiperparâmetros para o modelo proposto.

In [ ]:
print("Iniciando grid search...")
best_params, results_df = grid_search_word2vec(corpus, analogy_path, param_grid)

Iniciando grid search...
Treinando modelo com parâmetros: ('Skip-gram', 2, 5, 3, 3)
Parâmetros: ('Skip-gram', 2, 5, 3, True, 2, 3), Distância média: 0.2906
Parâmetros: ('Skip-gram', 2, 5, 3, True, 3, 3), Distância média: 0.3717
Parâmetros: ('Skip-gram', 2, 5, 3, True, 5, 3), Distância média: 0.4959
Parâmetros: ('Skip-gram', 2, 5, 3, False, 2, 3), Distância média: 0.4959
Parâmetros: ('Skip-gram', 2, 5, 3, False, 3, 3), Distância média: 0.4959
Parâmetros: ('Skip-gram', 2, 5, 3, False, 5, 3), Distância média: 0.4959
Treinando modelo com parâmetros: ('Skip-gram', 2, 5, 5, 3)
Parâmetros: ('Skip-gram', 2, 5, 5, True, 2, 3), Distância média: 0.3220
Parâmetros: ('Skip-gram', 2, 5, 5, True, 3, 3), Distância média: 0.3994
Parâmetros: ('Skip-gram', 2, 5, 5, True, 5, 3), Distância média: 0.5190
Parâmetros: ('Skip-gram', 2, 5, 5, False, 2, 3), Distância média: 0.5190
Parâmetros: ('Skip-gram', 2, 5, 5, False, 3, 3), Distância média: 0.5190
Parâmetros: ('Skip-gram', 2, 5, 5, False, 5, 3), Distância m

## Resultados

Após a execução do código, temos os resultados.

In [ ]:
print("Melhores parâmetros encontrados:", best_params)

Melhores parâmetros encontrados: ('CBOW', 2, 100, 3, True, 2, 3)


E para observar as demais avaliações dos modelos:

In [ ]:
import pandas as pd

# Filtrar as linhas onde PCA não foi aplicado
no_pca = results_df['PCA Aplicado'] == "Não Aplicado"

# Para as linhas onde PCA não foi aplicado, substituir os componentes por "-"
results_df.loc[no_pca, 'Componentes PCA'] = "-"

# Remover duplicatas considerando todas as colunas (com os "-" já ajustados)
results_df = results_df.drop_duplicates()

In [ ]:
results_df = results_df.sort_values(by=["Distância Média"])

In [ ]:
print("Melhores resultados detalhados:")
results_df.head(15)

Melhores resultados detalhados:


,Modelo,Contexto,Dimensão Embedding,Épocas,Workers,PCA Aplicado,Componentes PCA,Distância Média
576,CBOW,2,100,3,3,True,2,0.015313
696,CBOW,3,100,3,3,True,2,0.015499
816,CBOW,4,100,3,3,True,2,0.016060
936,CBOW,5,100,3,3,True,2,0.016917
582,CBOW,2,100,5,3,True,2,0.017851
577,CBOW,2,100,3,3,True,3,0.019645
697,CBOW,3,100,3,3,True,3,0.019893
817,CBOW,4,100,3,3,True,3,0.020398
937,CBOW,5,100,3,3,True,3,0.021197
702,CBOW,3,100,5,3,True,2,0.021291


In [ ]:
print("Piores resultados detalhados:")
results_df.tail(15)

Piores resultados detalhados:


,Modelo,Contexto,Dimensão Embedding,Épocas,Workers,PCA Aplicado,Componentes PCA,Distância Média
254,Skip-gram,4,5,7,3,True,5,0.682484
255,Skip-gram,4,5,7,3,False,Não Aplicado,0.682484
429,Skip-gram,5,25,10,3,False,Não Aplicado,0.706738
141,Skip-gram,3,5,10,3,False,Não Aplicado,0.712586
140,Skip-gram,3,5,10,3,True,5,0.712586
259,Skip-gram,4,5,10,3,True,3,0.720624
405,Skip-gram,5,20,10,3,False,Não Aplicado,0.729864
375,Skip-gram,5,5,7,3,False,Não Aplicado,0.742025
374,Skip-gram,5,5,7,3,True,5,0.742025
378,Skip-gram,5,5,10,3,True,2,0.753533


A partir desses resultados, tem-se que:

1. **Modelos (CBOW vs Skip-gram)**

CBOW apresenta valores significativamente menores para a distância média, indicando melhor desempenho na tarefa de prever analogias. Já o Skip-gram possui maiores distâncias médias, o que sugere menor precisão. Isso pode ocorrer por que o Skip-Gram, apesar de ser comumente mais eficaz em representar palavras raras, mas menos eficiente em tarefas gerais com vocabulário limitado.

2. **Tamanho do Contexto**

Para CBOW, o contexto 2 tem distâncias médias melhores (menores) em comparação aos demais tamanho de contexto.

3. **Dimensão do Embedding**

Para CBOW, embeddings de dimensão 100 consistentemente apresentam as menores distâncias médias, indicando que maior dimensionalidade captura melhor as relações semânticas. Dimensões mais baixas (como 5) parecem inadequadas para ambas as abordagens, com resultados menos precisos.

4. **Número de Épocas**

Para CBOW, o número de épocas 3 é suficiente para alcançar boas distâncias médias, sugerindo que o modelo converge rapidamente. Assim como ocorre com o Skip-Gram, é possível perceber que números maiores de época prejudicam o modelo.

* **PCA**

Os resultados foram consistentemente melhores quando houve a aplicação do PCA, especialmente com 2 componentes. Isso pode indicar que um espaço reduzido pode ser suficiente para representar as relações semânticas essenciais das analogias. Para o Skip-Gram, a aplicação do PCA não parece surtir tanto efeito em alguns resultados, de forma que seu uso se torna desnecessário.

Assim, tem-se que a melhor configuração é:
* Modelo: CBOW
* Tamanho do Contexto: 2
* Dimensão do Embedding: 100
* PCA: aplicado, com 2 componentes
* Épocas: 3

Essa configuração apresentação a menor distância média dentre os modelos avalidos, sendo ela **0.021291**.

## Avaliação com Modelo Nulo

Aqui, considera-se o modelo nulo como o modelo Word2Vec executado com os parâmetros padrão da implementação da biblioteca, dado o mesmo corpus. A intenção é identificar se há diferença entre os resultados calculados pelo modelo identificado pela Grid Search ou não.

In [ ]:
# Comparação com o modelo padrão
default_model = Word2Vec(
    sentences=corpus,
    vector_size=100,
    window=5,
    sg=0,  # 'CBOW' por padrão
    epochs=5,
    workers=3
)
default_avg_distance = evaluate_distance(default_model, analogy_path, False)
min_distance_model = results_df["Distância Média"].min()

print("\n--- Comparação entre Modelo Padrão e Melhor Modelo ---")
print(f"Distância Média do Modelo Padrão ('CBOW', 5, 100, 3, False, -, 3): {default_avg_distance:.4f}")
print(f"Melhores Parâmetros do Grid Search: {best_params}, Distância Média: {min_distance_model:.4f}")

if default_avg_distance > min_distance_model:
    improvement = ((default_avg_distance - min_distance_model) / default_avg_distance) * 100
    print(f"\nO modelo treinado com Grid Search possui distância mínima média {improvement:.2f}% menor que o modelo padrão.")
else:
    improvement = ((min_distance_model - default_avg_distance) / min_distance_model) * 100
    print(f"\nO modelo padrão possui distância mínima média {improvement:.2f}% menor que o modelo treinado com Grid Search.")


--- Comparação entre Modelo Padrão e Melhor Modelo ---
Distância Média do Modelo Padrão ('CBOW', 5, 100, 3, False, -, 3): 0.1257
Melhores Parâmetros do Grid Search: ('CBOW', 2, 100, 3, True, 2, 3), Distância Média: 0.0153

O modelo treinado com Grid Search possui distância mínima média 87.82% menor que o modelo padrão.


# Referências Bibliográficas

Jiménez, Á.B., Lázaro, J.L., Dorronsoro, J.R. (2007). Finding Optimal Model Parameters by Discrete Grid Search. In: Corchado, E., Corchado, J.M., Abraham, A. (eds) Innovations in Hybrid Intelligent Systems. Advances in Soft Computing, vol 44. Springer, Berlin, Heidelberg. https://doi.org/10.1007/978-3-540-74972-1_17

Kurita, T. (2020). Principal Component Analysis (PCA). In: Computer Vision. Springer, Cham. https://doi.org/10.1007/978-3-030-03243-2_649-1

Mahoney, Matt (2011). About the Test Data. Matt Mahoney. https://mattmahoney.net/dc/textdata.html

McCornick, Chris (2016). Word2Vec Resources. Chris McCormick. https://mccormickml.com/2016/04/27/word2vec-resources/

_ (2024). Word Embedding using Word2Vec. Geeks for Geeks. https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/